# Installation

### Install `pytorch_geometric`

In [ ]:
!pip install --upgrade torch-scatter

In [ ]:
!pip install --upgrade torch-sparse

In [ ]:
!pip install --upgrade torch-cluster

In [ ]:
!pip install --upgrade torch-spline-conv

In [ ]:
!pip install torch-geometric

### Install `proteinsolver`

In [ ]:
!pip install git+https://gitlab.com/ostrokach/proteinsolver.git

# Imports

In [22]:
import tempfile

import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [23]:
import proteinsolver
import proteinsolver.datasets

In [24]:
%load_ext autoreload
%autoreload 2

# Datasets

## `SudokuDataset`

In [25]:
sudoku_dataset_train = proteinsolver.datasets.SudokuDataset("train")

In [26]:
sudoku_dataset_train[0]

Data(edge_index=[2, 1701], label=[1], x=[81, 1], y=[81, 1])

In [27]:
sudoku_dataset_valid = proteinsolver.datasets.SudokuDataset("valid")

In [28]:
sudoku_dataset_valid[0]

Data(edge_index=[2, 1701], label=[1], x=[81, 1], y=[81, 1])

## `TUDataset`

In [29]:
from torch_geometric.datasets import TUDataset

In [30]:
tu_dataset = TUDataset(root=tempfile.gettempdir() + '/ENZYMES', name='ENZYMES')

In [31]:
tu_dataset[0]

Data(edge_index=[2, 168], x=[37, 3], y=[1])

# Models

In [32]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [33]:
net = GCNConv(tu_dataset.num_features, 16)

In [43]:
net = GCNConv(9, 16)

y_onehot = torch.FloatTensor(81, 9)

data = sudoku_dataset_train[0]
y_onehot.zero_()
y_onehot.scatter_(1, data.y - 1, 1)
net.forward(y_onehot, data.edge_index).shape

torch.Size([81, 16])

In [39]:
data.x.shape

torch.Size([81, 1])

In [16]:
data

Data(edge_index=[2, 168], x=[37, 3], y=[1])

In [7]:
class Net(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super().__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(tu_dataset.num_features, tu_dataset.num_classes).to(device)
data = tu_dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


AttributeError: 'Data' object has no attribute 'train_mask'

In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))
